## Import libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

## Data location

In [2]:
data_loc = "S://Docs//Personal//MAEVE//Data//"

## Pull data

In [3]:
path = data_loc + "BTC_price_1h.csv"
df = pd.read_csv(path)

print(f"Data shape: {df.shape}")

print(f"Date range: {df.Datetime.min()} - {df.Datetime.max()}")

Data shape: (17017, 7)
Date range: 2021-02-01 00:00:00+00:00 - 2023-01-20 13:00:00+00:00


## Custom functions

In [4]:
def calc_MA(df, timeperiod):
    df[f'MA{timeperiod}'] = df['Close'].rolling(window=timeperiod).mean()
    return df

## Calculate moving averages

In [5]:
# Calculate MA
MALst = [12, 20, 24, 30, 40, 48, 50, 60, 100, 200]

for MA in MALst:
    df = calc_MA(df, MA)

df.shape


(17017, 17)

In [7]:
df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume,MA12,MA20,MA24,MA30,MA40,MA48,MA50,MA60,MA100,MA200
0,2021-02-01 00:00:00+00:00,33114.578125,33114.578125,32384.228516,32551.958984,32551.958984,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01 01:00:00+00:00,32556.548828,33470.128906,32556.548828,33450.535156,33450.535156,1724944384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-01 02:00:00+00:00,33448.257812,33769.785156,33331.187500,33655.015625,33655.015625,458350592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-01 03:00:00+00:00,33654.921875,33712.281250,33531.136719,33562.886719,33562.886719,782512128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-01 04:00:00+00:00,33567.050781,33815.316406,33484.789062,33563.953125,33563.953125,1084145664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def log_trade(price, pos, cash, sats):
    
    df = pd.DataFrame()
    
    df['price'] = [price]
    df['tradeType'] = [pos]
    df['cash'] = [cash]
    df['sats'] = [sats]
    
    return df

In [15]:
# Initialize the strategy variables
current_position = None  # "buy" or "sell"
cash = 100.00  # Starting cash
sats = 0  # Starting BTC

trades_df = pd.DataFrame()

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
    # Calculate the MA20 and MA40
    MA20 = row['MA20']
    MA40 = row['MA40']

    # Check if the MA20 is higher than the MA40
    if MA20 > MA40:
        # If we're not currently holding any BTC, buy BTC
        if current_position != "buy":
            sats = round(cash / row['Close'], 8)
            cash = 0
            current_position = "buy"
            trades_df = pd.concat([trades_df, log_trade(row['Close'], current_position, cash, sats)])
            
    # Check if the MA20 is lower than the MA40
    elif MA20 < MA40:
        # If we're currently holding BTC, sell
        if current_position == "buy":
            cash = round(sats * row['Close'], 2)
            sats = 0
            current_position = "sell"
            trades_df = pd.concat([trades_df, log_trade(row['Close'], current_position, cash, sats)])

# Print the final cash and sats value
print("Final cash value:", cash)
print("Final sats value:", sats)


Final cash value: 0
Final sats value: 0.00301445


In [16]:
trades_df

,price,tradeType,cash,sats
0,34678.085938,buy,0.00,0.002884
0,37264.953125,sell,107.46,0.000000
0,37727.167969,buy,0.00,0.002848
0,38789.417969,sell,110.49,0.000000
0,43241.972656,buy,0.00,0.002555
...,...,...,...,...
0,17422.755859,buy,0.00,0.003048
0,20791.261719,sell,63.37,0.000000
0,20879.103516,buy,0.00,0.003035
0,20949.736328,sell,63.58,0.000000
